# 🧪 Тестовое задание для аналитика


Добро пожаловать! Это тестовое задание поможет нам оценить ваше знание Python (pandas), SQL, работы с базой данных, а также базовые навыки работы с Docker и сервером.

📁 Данные: в папке находится файл `orders_report.xlsx`, содержащий «грязные» данные из CRM.

Заполните ячейки ниже.


## 1. Обработка Excel-файла


🔹 **Задание:**
- СДЕЛАТЬ EDA АНАЛИЗ;
- ОБРАБОТАТЬ ДАННЫЕ, ПОЧИСТИТЬ И НЕ ПОТЕРЯТЬ ВАЖНЫЕ ДАННЫЕ

- ПОСЧИТАТЬ:
  - количество уникальных заказов;
  - топ-3 товара по выручке;
  - средний чек по каналу продаж.


✏️ **Ответ:**

In [ ]:
# Ответ


### 2. Загрузка данных в БД (**ОТРАЗИТЬ СИНТАКСИС И ШАГИ, К САМОЙ БД ПОДКЛЮЧАТЬСЯ НЕ НУЖНО!**)

### Задание:

    - Подключиться к БД 
        Пользователь - testuser
        Пароль - password  
        БД - testdb
        ХОСТ - localhost
        ПОРТ - 5432

    - СОЗДАТЬ таблицу "orders_cleaned" и загрузить датафрейм из 1-го задания
    - Завершить подключение


✏️ **Ответ:**

In [ ]:
# ОТВЕТ

## 3. SQL-запросы


Допустим, в БД появилась таблица `orders_cleaned`.

✍️ Напишите SQL-запросы:

1. Выручка по каждому товару за последний месяц.
2. Клиенты, сделавшие более 3 заказов.
3. Заказы, в которых одновременно есть товары X и Y.


In [ ]:
# ОТВЕТ 

## 4. Docker и серверные команды


✏️ Ответьте на следующие вопросы:

1. Как упаковать этот скрипт в Docker-контейнер?
2. Какие команды используете для:
   - перезапуска контейнера
   - просмотра логов
   - остановки зависшего процесса
3. Как бы вы настроили запуск скрипта каждый день в 8:00 через cron?
4. Что делать, если команда на сервере "зависла"? Как дебажить?


In [ ]:
# ОТВЕТ 

---
📨 Спасибо за выполнение задания!